In [4]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

from scipy.sparse import coo_matrix
import numpy as np

In [5]:
from rough.code.gcn_layer import GCNLayer
from rough.code.utils import adjust_tensor_size, normalize_adjacency

# Example usage:
num_nodes = 1000
num_node_features = 1
num_output_features = 512
batch_size = 32

# Create a random feature matrix (num_nodes x num_node_features)
random_node_features = torch.rand(batch_size, num_nodes - 10, num_node_features)
node_features = adjust_tensor_size(random_node_features, num_nodes, num_node_features)

# Create a random adjacency matrix
adjacency_matrix = torch.randint(0, 2, size=(batch_size, num_nodes, num_nodes))
# Iterate over the diagonal elements
# for i in range(num_nodes):
#     if adjacency_matrix[i, i] == 0:
#         adjacency_matrix[i, i] = 1

normalized_adjacency_matrix = normalize_adjacency(adjacency_matrix, num_nodes, batch_size)

# Instantiate the GCN layer
gcn_layer = GCNLayer(in_features=num_node_features, out_features=num_output_features)

# Perform a forward pass
print(node_features.shape)
print(normalized_adjacency_matrix.shape)
gcn_output = gcn_layer(node_features, normalized_adjacency_matrix)

print("\n---Sample Inputs---")
print("Number of nodes: ", num_nodes)
print("Number of features: ", num_node_features)
print("Number of outputs: ", num_output_features)
print("\n---Matrices---")
print("Random Node Features: ", random_node_features.shape)
print("Node features:", node_features.shape)
print("Adjacency matrix:", adjacency_matrix.shape)
print("Normalized Adjacency matrix:", normalized_adjacency_matrix.shape)
print("GCN In Features:", num_node_features)
print("GCN Out Features:", num_output_features)
print("\n---GCN Model---")
print("GCN Layer Representation:", gcn_layer.__repr__)
print("GCN Output Shape:", gcn_output.shape)

torch.Size([32, 1000, 1])
torch.Size([32, 1000, 1000])

---Sample Inputs---
Number of nodes:  1000
Number of features:  1
Number of outputs:  512

---Matrices---
Random Node Features:  torch.Size([32, 990, 1])
Node features: torch.Size([32, 1000, 1])
Adjacency matrix: torch.Size([32, 1000, 1000])
Normalized Adjacency matrix: torch.Size([32, 1000, 1000])
GCN In Features: 1
GCN Out Features: 512

---GCN Model---
GCN Layer Representation: <bound method GCNLayer.__repr__ of GCNLayer (1 -> 512)>
GCN Output Shape: torch.Size([32, 1000, 512])


In [6]:
import torch.optim as optim

# For binary classification, targets could be a 1D tensor of size (batch_size,)
targets = torch.randint(0, 2, (batch_size,))

# Define the model
model = GCNLayer(in_features=num_node_features, out_features=2)

# Loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()  # Clear gradients

    # Forward pass
    logits = model(node_features, normalized_adjacency_matrix)
    # Since we have 2 output features, we use CrossEntropyLoss and gather logits for each graph
    logits = logits.mean(1)  # Mean pooling
    loss = criterion(logits, targets)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    # Print loss every 10 epochs
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 0.8817738890647888
Epoch 10, Loss: 0.8024495244026184
Epoch 20, Loss: 0.7449772357940674
Epoch 30, Loss: 0.709841251373291
Epoch 40, Loss: 0.6927292346954346
Epoch 50, Loss: 0.6866394877433777
Epoch 60, Loss: 0.68534916639328
Epoch 70, Loss: 0.6853305697441101
Epoch 80, Loss: 0.685374915599823
Epoch 90, Loss: 0.6853427886962891


In [ ]:
# Notes:
# 1. Take the imports
# 2. Take and Modify Cell 3 which shows how to call the model with the given inputs and preprocessing utilities
# 3. Cell 6 is just to test. Not required to take this cell.